# 🪙 G-Research Crypto - Starter LGBM Pipeline
![](https://storage.googleapis.com/kaggle-competitions/kaggle/30894/logos/header.png)

Based on:  https://www.kaggle.com/julian3833/g-research-starter-lgbm-pipeline-lb-0-164
 

### Just a simple pipeline going from zero to a valid submission

We train one `LGBMRegressor` for each asset over a very simple set of features, we get the predictions correctly using the iterator and submit. 


## Please upvote if you find this useful!

## References:
* [Detailed API Introduction](https://www.kaggle.com/sohier/detailed-api-introduction)
* [Basic Submission Template](https://www.kaggle.com/sohier/basic-submission-template)
* [Tutorial to the G-Research Crypto Competition](https://www.kaggle.com/cstein06/tutorial-to-the-g-research-crypto-competition)



# Import and load dfs

References: [Tutorial to the G-Research Crypto Competition](https://www.kaggle.com/cstein06/tutorial-to-the-g-research-crypto-competition)

In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
import gresearch_crypto
from tensorflow.python.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.python.keras.layers import Dropout, LSTM, Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential


TRAIN_CSV = '/kaggle/input/g-research-crypto-forecasting/train.csv'
ASSET_DETAILS_CSV = '/kaggle/input/g-research-crypto-forecasting/asset_details.csv'

In [ ]:
df_train = pd.read_csv(TRAIN_CSV)#,nrows=9234567)
# df_train = df_train.sort_values(["timestamp","Asset_ID"]) # needed if adding lag features (problematic for target)
df_train.head()

In [ ]:
df_asset_details = pd.read_csv(ASSET_DETAILS_CSV).sort_values("Asset_ID")
df_asset_details

# Training

## Utility functions to train a model for one asset

In [ ]:
# Two new features from the competition tutorial
def upper_shadow(df):
    return df['High'] - np.maximum(df['Close'], df['Open'])

def lower_shadow(df):
    return np.minimum(df['Close'], df['Open']) - df['Low']

# A utility function to build features from the original df
# It works for rows to, so we can reutilize it.
def get_features(df,row=False):
    df_feat = df[["Asset_ID",'Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP',"timestamp"]].copy()
    df_feat['Upper_Shadow'] = upper_shadow(df_feat)
    df_feat['Lower_Shadow'] = lower_shadow(df_feat)
    
    ## Ad dsome more feats
    df_feat["high_div_low"] = df_feat["High"]/df_feat["Low"]
    df_feat["open_sub_close"] = df_feat["Open"]-df_feat["Close"]

    ## possible seasonality, datetime  features (unlikely to me meaningful, given very short time-frames)
    ### to do: add cyclical features for seasonality
    times = pd.to_datetime(df_feat["timestamp"],unit="s",infer_datetime_format=True)
    if row:
        df_feat["hour"] = times.hour  # .dt
        df_feat["dayofweek"] = times.dayofweek 
        df_feat["day"] = times.day 
    else:
        df_feat["hour"] = times.dt.hour  # .dt
        df_feat["dayofweek"] = times.dt.dayofweek 
        df_feat["day"] = times.dt.day 
    df_feat.drop(columns=["time"],errors="ignore",inplace=True)  # keep original epoch time, drop string

    
    ## todo: features of othet crypto assets in same time period (pivot table + avoid target col)
    ## Lag of target (be careful to avoid leak? Is this valid for test data (in terms of input format?))
    ## more features
    ### time window/time series features
    return df_feat

def get_Xy_and_model_for_asset(df_train, asset_id):
    df = df_train[df_train["Asset_ID"] == asset_id]
    
    # TODO: Try different features here!
    df_proc = get_features(df)
    df_proc['y'] = df['Target']
    df_proc = df_proc.dropna(how="any")
    df_proc = df_proc[~ df_proc.isin([np.nan, np.inf, -np.inf]).any(1)]
    X = df_proc.drop("y", axis=1)
    X = X.values
    y = df_proc["y"].values
    X_resampled = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(128, input_shape=(X_resampled.shape[1], X_resampled.shape[2])))
    model.add(LeakyReLU(alpha=0.03))
    model.add(BatchNormalization())
    model.add(Dense(16))
    model.add(LeakyReLU(alpha=0.03))
    model.add(BatchNormalization())
    model.add(Dense(1))
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss="mean_squared_error", optimizer=optimizer)
    model.fit(X_resampled, y, epochs=1, batch_size=512, verbose=1, shuffle=True)

    return X, y, model

## Loop over all assets

In [ ]:
%%time
Xs = {}
ys = {}
models = {}

for asset_id, asset_name in zip(df_asset_details['Asset_ID'], df_asset_details['Asset_Name']):
    print(f"Training model for {asset_name:<16} (ID={asset_id:<2})")
    X, y, model = get_Xy_and_model_for_asset(df_train, asset_id)    
    Xs[asset_id], ys[asset_id], models[asset_id] = X, y, model

In [ ]:
%%time
# Check the model interface
x = get_features(df_train.iloc[0],row=True).values
X_resampled = x.reshape(1, 1, x.shape[0])
y_pred = models[0].predict([X_resampled])
y_pred[0]

# Predict & submit

References: [Detailed API Introduction](https://www.kaggle.com/sohier/detailed-api-introduction)

Something that helped me understand this iterator was adding a pdb checkpoint inside of the for loop:

```python
import pdb; pdb.set_trace()
```

See [Python Debugging With Pdb](https://realpython.com/python-debugging-pdb/) if you want to use it and you don't know how to.


In [ ]:
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

for i, (df_test, df_pred) in enumerate(iter_test):
    for j , row in df_test.iterrows():
        model = models[row['Asset_ID']]
        x_test = get_features(row,row=True).values
        X_resampled = x_test.reshape(1, 1, x_test.shape[0])
        y_pred = model.predict([X_resampled])[0]
        
        df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = y_pred
        
        # Print just one sample row to get a feeling of what it looks like
        if i == 0 and j == 0:
            display(x_test)

    # Display the first prediction dataframe
    if i == 0:
        display(df_pred)

    # Send submissions
    env.predict(df_pred)